# Guided Project: Answering Business Questions Using SQL

We'll use the following code to connect our Jupyter Notebook to our database file:

In [7]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## Data overview

Below query shows information about the tables and view of the database. We will establish a schema which will help us to to join the different tables.

In [8]:
%%sql

SELECT 
      name, 
      type
FROM sqlite_master
WHERE type IN ("table", "view")

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Selecting New Albums for Our Online Store

The Chinook record store has signed a new record label and we have been asked to select the first three albums that will be added to the store, from a list of four. 
All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce: Hip-Hop, Punk, Pop, Blues.

Thus, with the below queries we will try and answer, which are the most popular genres that sell in the USA. We will start by joining multiple tables together, thus we will have the necessary columns to calculate.

In [9]:
%%sql

WITH 
    tracks_usa AS
                (
                 SELECT g.name AS genre, 
                 COUNT(il.quantity) AS tracks_sold, 
                 c.country
                 FROM invoice_line AS il 
                 LEFT JOIN invoice AS i ON i.invoice_id = il.invoice_id
                 LEFT JOIN track AS t ON t.track_id = il.track_id
                 LEFT JOIN genre AS g ON g.genre_id = t.genre_id 
                 LEFT JOIN customer AS c ON c.customer_id = i.customer_id
                 WHERE c.country = 'USA'
                 GROUP BY t.genre_id
                 ORDER BY 2 DESC
                )
SELECT *,
CAST(tracks_sold AS FLOAT)  / (SELECT SUM(tracks_sold) FROM tracks_usa) * 100 AS percentage_tracks_sold
FROM tracks_usa;

Done.


genre,tracks_sold,country,percentage_tracks_sold
Rock,561,USA,53.37773549000951
Alternative & Punk,130,USA,12.369172216936251
Metal,124,USA,11.798287345385347
R&B/Soul,53,USA,5.042816365366318
Blues,36,USA,3.425309229305423
Alternative,35,USA,3.3301617507136063
Latin,22,USA,2.093244529019981
Pop,22,USA,2.093244529019981
Hip Hop/Rap,20,USA,1.9029495718363463
Jazz,14,USA,1.3320647002854424


Based on the above table we can conclude that the three genres which are selling the most in USA are (in descending order):
* Rock - 53% 
* Alternative & Punk - 12%
* Metal - 11%

We were offered four different artists in four type of genres - Hip-Hop, Punk, Pop, and Blues. Based on the online song sales, the least popular genre out of these four is Hip-Hop with only 1.9% sales in USA. Thus, we will include the artists who are in the remainder three genre categories because they sell much better. We will purchase:

* Red Tone - Punk
* Meteor and the Girls - Pop
* Slim Jim Bites - Blues

We should still consider adding more artists from the Rock genre as it accounts for just over a half of the total sales with 53%.

## Analyzing Sales Support Agents Performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We are going to analyze the customer purchases belonging to each employee to see if any sales support ort agent is performing better or worse than the others.


In [10]:
%%sql

SELECT e.first_name ||" "|| e.last_name AS employee_name,
e.title, 
e.hire_date,
SUM(i.total) AS amount_total_sales
FROM employee AS e
INNER JOIN customer AS c ON c.support_rep_id = e.employee_id
INNER JOIN invoice AS i ON i.customer_id = c.customer_id
GROUP BY employee_name;

Done.


employee_name,title,hire_date,amount_total_sales
Jane Peacock,Sales Support Agent,2017-04-01 00:00:00,1731.5100000000039
Margaret Park,Sales Support Agent,2017-05-03 00:00:00,1584.0000000000034
Steve Johnson,Sales Support Agent,2017-10-17 00:00:00,1393.920000000002


As seen from the above table, there is about 100 - 150$ difference in the total number of sales between the three employees.
Jane is on top with the most sales, second is Margaret, and lastly, we have Steve. This slight difference in sales can be explained by the different hire date of the three employees. As expected, Jane was hired first, and a month after that Margaret was hired, thus, the difference is not that big. Five months later they have hired Steve, who is last in terms of sales. 
Overall, there is a small difference between their sales performance.

## Analyzing Sales by Country

We are going to analyze sales data for customers from each country.

In [11]:
%%sql 

WITH country_or_other AS
    (
     SELECT 
           CASE 
               WHEN (
                     SELECT
                           COUNT(*)
                       FROM customer
                      WHERE country = c.country
                     ) = 1 THEN "Other"
               ELSE c.country
           END AS country,
           c.customer_id,
           il.*
     FROM invoice_line AS il
     INNER JOIN invoice AS i ON i.invoice_id = il.invoice_id
     INNER JOIN customer AS c ON c.customer_id = i.customer_id      
    )
    
SELECT 
      country,
      customers,
      total_sales,
      average_order,
      customer_lifetime_value
FROM
    (
     SELECT 
           country,
           COUNT(DISTINCT customer_id) AS customers,
           SUM(unit_price) AS total_sales,
           SUM(unit_price) / COUNT(DISTINCT customer_id) AS customer_lifetime_value,
           SUM(unit_price) / COUNT(DISTINCT invoice_id) AS average_order,
           CASE
               WHEN country = "Other" THEN 1
               ELSE 0
           END AS sort
       FROM country_or_other
       GROUP BY country
       ORDER BY sort ASC, total_sales DESC
    );

Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
India,2,183.1500000000002,8.72142857142858,91.5750000000001
Other,15,1094.9400000000085,7.448571428571486,72.99600000000056


We have calculated the data for each country based on the:

* total number of customers
* total value of sales
* average value of sales per customer
* average order value

As there are number of countries in our database with only one customer, we have grouped these countries in "Other" category.

Based on the data there seems to be an opportunity in the following countries:

* Czech Republic
* United Kingdom
* India

Our data sample for all countries is low, and this should be taken into consideration. When deciding on how much money to spend on new marketing campaigns. Because our data does not give us a detailed picture on which country to advertise more it is better to do small marketing campaigns in all these countries. And also, to collect new data and to ensure these trends hold true with the new customers

## Albums vs Individual Tracks

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

* purchase a whole album

* purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We will try to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In [12]:
%%sql

WITH invoice_first_track AS
    (SELECT 
           il.invoice_id,
           MIN(il.track_id) AS first_track_id
     FROM invoice_line AS il
     GROUP BY 1
    )

SELECT 
      album_purchase,
      COUNT(invoice_id) AS number_of_invoices,
      CAST(COUNT(invoice_id) AS FLOAT) / (SELECT COUNT(*) FROM invoice) AS percent
FROM (
      SELECT 
            ifs.*,
            CASE 
                WHEN
                    (
                     SELECT t.track_id FROM track AS t
                      WHERE t.album_id = (
                                          SELECT t2.album_id FROM track AS t2
                                           WHERE t2.track_id = ifs.first_track_id
                     
                                         )
                     EXCEPT
                     
                     SELECT il2.track_id FROM invoice_line AS il2
                      WHERE il2.invoice_id = ifs.invoice_id                        
                    ) IS NULL
                 AND
                    (
                    SELECT il2.track_id FROM invoice_line AS il2
                     WHERE il2.invoice_id = ifs.invoice_id
                    
                    EXCEPT
                    
                    SELECT t.track_id FROM track AS t
                     WHERE t.album_id = (
                                        SELECT t2.album_id FROM track AS t2
                                         WHERE t2.track_id = ifs.first_track_id
                                        )
                    ) IS NULL
                THEN "yes"
                ELSE "no"
              END AS "album_purchase"
      FROM invoice_first_track AS ifs
     )
GROUP BY album_purchase;

Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


Last query that we have run shows that album purchases account for about 18.6% of total purchases. Thus, we can recommend purchasing whole albums rather than selected tracks from record companies, otherwise they could lose revenue.